In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_context("poster")
%matplotlib inline

In [2]:
data_dir = "/Users/maxverbiest/PhD/projects/genotype_strs/data/fotsing_data/"
df = pd.read_csv(data_dir + "fotsing_etal_data_sheet_hg38_tral_panel.tsv", header = None, sep="\t")
df['tmp_id'] = df.apply(lambda row: f"{row[12]}_{row[13]}", axis = 1)
df.drop([12, 13, 14, 15, 17, 18], axis=1, inplace=True)
df.columns = [
  "chr",
  "start",
  "end",
  "gene_id",
  "gene_name",
  "tissue_hits",
  "beta_max",
  "tissue_info",
  "pval",
  "CAVIAR_score",
  "unit_fw",
  "unit_rv",
  "unit_panel",
  "panel_overlap",
  "tmp_id"
]

In [3]:
df.to_csv(f"{data_dir}fotsing_etal_tmp_id_mapping.tsv", sep="\t", index=False)

In [4]:
print(df.info())
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22393 entries, 0 to 22392
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   chr            22393 non-null  object 
 1   start          22393 non-null  int64  
 2   end            22393 non-null  int64  
 3   gene_id        22393 non-null  object 
 4   gene_name      22393 non-null  object 
 5   tissue_hits    22393 non-null  int64  
 6   beta_max       22393 non-null  float64
 7   tissue_info    22393 non-null  object 
 8   pval           22393 non-null  float64
 9   CAVIAR_score   22347 non-null  float64
 10  unit_fw        22393 non-null  object 
 11  unit_rv        22393 non-null  object 
 12  unit_panel     22393 non-null  object 
 13  panel_overlap  22393 non-null  int64  
 14  tmp_id         22393 non-null  object 
dtypes: float64(3), int64(4), object(8)
memory usage: 2.6+ MB
None


start           end   tissue_hits      beta_max          pval  \
count  2.239300e+04  2.239300e+04  22393.000000  22393.000000  2.239300e+04   
mean   7.453439e+07  7.453442e+07      1.981155      0.004186  3.152115e-04   
std    5.566473e+07  5.566473e+07      2.146705      0.319096  6.417859e-04   
min    4.949300e+04  4.950400e+04      1.000000     -0.842330  1.020000e-69   
25%    3.262554e+07  3.262557e+07      1.000000     -0.271403  2.630000e-07   
50%    6.058528e+07  6.058533e+07      1.000000      0.180656  4.800000e-05   
75%    1.095858e+08  1.095858e+08      2.000000      0.273124  3.961950e-04   
max    2.489043e+08  2.489043e+08     17.000000      0.814063  2.584264e-02   

       CAVIAR_score  panel_overlap  
count  2.234700e+04   22393.000000  
mean   7.148528e-02      19.111017  
std    1.365815e-01      10.788728  
min    7.950000e-16       1.000000  
25%    9.636540e-03      11.000000  
50%    2.422930e-02      15.000000  
75%    6.501635e-02      24.000000  
max    1.000000e+00      91.000000

In [5]:
df.head()

chr    start      end          gene_id gene_name  tissue_hits  beta_max  \
0  chr1   950660   950671  ENSG00000187583   PLEKHN1            2 -0.226886   
1  chr1   950660   950671  ENSG00000187961    KLHL17            1 -0.195615   
2  chr1   950660   950671  ENSG00000188976     NOC2L            5  0.312156   
3  chr1   965304   965346  ENSG00000187642  C1orf170            1 -0.252924   
4  chr1  1002183  1002202  ENSG00000187608     ISG15            2  0.208372   

                                         tissue_info          pval  \
0  Cells-Transformedfibroblasts_-0.22_0.01;Nerve-...  2.068940e-04   
1                        Esophagus-Mucosa_-0.20_0.01  1.733056e-03   
2  Artery-Tibial_0.19_0.01;Lung_0.20_0.07;Nerve-T...  4.350000e-08   
3                    Esophagus-Muscularis_-0.25_0.01  1.911650e-04   
4     Artery-Tibial_0.21_0.05;Nerve-Tibial_0.19_0.08  6.725450e-04   

   CAVIAR_score unit_fw unit_rv unit_panel  panel_overlap        tmp_id  
0      0.040808      AC      GT         CA             11   chr1_950660  
1      0.010497      AC      GT         CA             11   chr1_950660  
2      0.103870      AC      GT         CA             11   chr1_950660  
3      0.012421    ATTT    AAAT       TTTA             11   chr1_965316  
4      0.082017       T       A          T             19  chr1_1002183

In [6]:
def get_consensus_unit(unit: str) -> str:
    """ Determine all circular permutations of input unit, sort alphabetically and return the 
    first permutation. e.g. TTTA -> ['TTTA', 'TTAT', 'TATT', 'ATTT'] -> ATTT
    """
    unit2 = unit + unit
    permutations = []

    for i in range(0, len(unit)):
         permutations.append( unit2[i : i+len(unit)] )
            
    return sorted(permutations)[0]

In [7]:
df.loc[:, 'unit_panel'] = df.apply(lambda row: get_consensus_unit(row.unit_panel), axis=1)

In [8]:
df.head()

chr    start      end          gene_id gene_name  tissue_hits  beta_max  \
0  chr1   950660   950671  ENSG00000187583   PLEKHN1            2 -0.226886   
1  chr1   950660   950671  ENSG00000187961    KLHL17            1 -0.195615   
2  chr1   950660   950671  ENSG00000188976     NOC2L            5  0.312156   
3  chr1   965304   965346  ENSG00000187642  C1orf170            1 -0.252924   
4  chr1  1002183  1002202  ENSG00000187608     ISG15            2  0.208372   

                                         tissue_info          pval  \
0  Cells-Transformedfibroblasts_-0.22_0.01;Nerve-...  2.068940e-04   
1                        Esophagus-Mucosa_-0.20_0.01  1.733056e-03   
2  Artery-Tibial_0.19_0.01;Lung_0.20_0.07;Nerve-T...  4.350000e-08   
3                    Esophagus-Muscularis_-0.25_0.01  1.911650e-04   
4     Artery-Tibial_0.21_0.05;Nerve-Tibial_0.19_0.08  6.725450e-04   

   CAVIAR_score unit_fw unit_rv unit_panel  panel_overlap        tmp_id  
0      0.040808      AC      GT         AC             11   chr1_950660  
1      0.010497      AC      GT         AC             11   chr1_950660  
2      0.103870      AC      GT         AC             11   chr1_950660  
3      0.012421    ATTT    AAAT       ATTT             11   chr1_965316  
4      0.082017       T       A          T             19  chr1_1002183

In [9]:
df_str_info = pd.read_csv(
    "/Users/maxverbiest/PhD/data/str_panels/tral_and_perf_panel_meta_info.tsv",
    sep="\t"
)
df_patient_variation = pd.read_csv(
    "/Users/maxverbiest/PhD/projects/genotype_strs/results/oxana_variation/patient_variation_wide.tsv",
    sep="\t"
)

In [10]:
df_patient_variation.head()

patient        tmp_id  allele_A_tumor  allele_B_tumor  \
0  TCGA-AA-3489   chr1_187770               4               4   
1  TCGA-AA-3489   chr1_971311               3               3   
2  TCGA-AA-3489  chr1_1043875               5               5   
3  TCGA-AA-3489  chr1_1051937               4               4   
4  TCGA-AA-3489  chr1_1090878               4               4   

   allele_A_healthy  allele_B_healthy  period  ref repeat_id  patient_len_diff  
0                 4                 4       2    4   1099253                 0  
1                 3                 3       4    3   1099283                 0  
2                 5                 5       2    5   1099359                 0  
3                 4                 4       2    4   1099381                 0  
4                 4                 4       2    4   1099469                 0

In [11]:
df_patient_variation_filt = (
    df_patient_variation
        .merge(df_str_info, how="left", on="tmp_id", suffixes=(None, "_tmp"))
        .query("not in_segdup and neighbour_type == 'no_neighbour' and period > 1")
        .loc[:, df_patient_variation.columns] 
)


df_patient_variation_filt = (
    df_patient_variation_filt
        .groupby('patient')
        .filter(lambda x: len(x) > 5000)
        .reset_index()
        .drop("index", axis=1)
)

print(df_patient_variation_filt['patient'].nunique())
df_patient_variation_filt.describe()

88


allele_A_tumor  allele_B_tumor  allele_A_healthy  allele_B_healthy  \
count    2.106935e+06    2.106935e+06      2.106935e+06      2.106935e+06   
mean     4.562266e+00    4.577307e+00      4.564733e+00      4.578475e+00   
std      2.171345e+00    2.269925e+00      2.179259e+00      2.283358e+00   
min      1.000000e+00    1.000000e+00      1.000000e+00      1.000000e+00   
25%      4.000000e+00    4.000000e+00      4.000000e+00      4.000000e+00   
50%      4.000000e+00    4.000000e+00      4.000000e+00      4.000000e+00   
75%      5.000000e+00    5.000000e+00      5.000000e+00      5.000000e+00   
max      3.600000e+01    5.800000e+01      3.900000e+01      3.850000e+02   

             period           ref  patient_len_diff  
count  2.106935e+06  2.106935e+06      2.106935e+06  
mean   2.621197e+00  4.584560e+00      6.603668e-02  
std    9.908131e-01  2.274640e+00      6.541689e-01  
min    2.000000e+00  3.000000e+00      0.000000e+00  
25%    2.000000e+00  4.000000e+00      0.000000e+00  
50%    2.000000e+00  4.000000e+00      0.000000e+00  
75%    3.000000e+00  5.000000e+00      0.000000e+00  
max    6.000000e+00  5.000000e+01      3.600000e+02

In [12]:
df_patient_variation_filt.head()

patient        tmp_id  allele_A_tumor  allele_B_tumor  \
0  TCGA-AA-3489   chr1_971311               3               3   
1  TCGA-AA-3489  chr1_1043875               5               5   
2  TCGA-AA-3489  chr1_1051937               4               4   
3  TCGA-AA-3489  chr1_1090878               4               4   
4  TCGA-AA-3489  chr1_1182416               4               4   

   allele_A_healthy  allele_B_healthy  period  ref repeat_id  patient_len_diff  
0                 3                 3       4    3   1099283                 0  
1                 5                 5       2    5   1099359                 0  
2                 4                 4       2    4   1099381                 0  
3                 4                 4       2    4   1099469                 0  
4                 4                 4       2    4   1099502                 0

In [23]:
df_in_estr = (
    df_patient_variation_filt[
        df_patient_variation_filt["tmp_id"]
            .isin(df["tmp_id"].unique())
    ]
)

print(df_in_estr.shape)
df_in_estr.head()

(13658, 10)


patient          tmp_id  allele_A_tumor  allele_B_tumor  \
13   TCGA-AA-3489    chr1_1752910              10              10   
459  TCGA-AA-3489   chr1_55058667              16              16   
497  TCGA-AA-3489   chr1_66643729               7               7   
646  TCGA-AA-3489  chr1_101021818              21              21   
673  TCGA-AA-3489  chr1_109759146              16              18   

     allele_A_healthy  allele_B_healthy  period  ref repeat_id  \
13                 10                10       3    9   1100327   
459                18                19       2   26   1142870   
497                 7                 7       2    7   1150529   
646                22                22       2   21   1166924   
673                15                17       2   19   1169780   

     patient_len_diff  
13                  0  
459                 5  
497                 0  
646                 2  
673                 2

In [14]:
df_in_estr_variable = (
    df_in_estr[df_in_estr["patient_len_diff"] > 0]
)
df_in_estr_variable.head()

patient          tmp_id  allele_A_tumor  allele_B_tumor  \
459   TCGA-AA-3489   chr1_55058667              16              16   
646   TCGA-AA-3489  chr1_101021818              21              21   
673   TCGA-AA-3489  chr1_109759146              16              18   
1799  TCGA-AA-3489   chr2_69962200              14              14   
1966  TCGA-AA-3489  chr2_111968092              15              15   

      allele_A_healthy  allele_B_healthy  period  ref repeat_id  \
459                 18                19       2   26   1142870   
646                 22                22       2   21   1166924   
673                 15                17       2   19   1169780   
1799                13                14       2   16    276206   
1966                18                18       2   15    294491   

      patient_len_diff  
459                  5  
646                  2  
673                  2  
1799                 1  
1966                 6

In [15]:
df_in_estr_variable = (
    df_in_estr_variable
        .merge(df.loc[:, ['tmp_id', 'gene_name', 'beta_max', 'CAVIAR_score']], on="tmp_id")
)
print(df_in_estr_variable.shape)
df_in_estr_variable.head()

(4229, 13)


patient         tmp_id  allele_A_tumor  allele_B_tumor  \
0  TCGA-AA-3489  chr1_55058667              16              16   
1  TCGA-A6-2682  chr1_55058667              18              19   
2  TCGA-A6-2671  chr1_55058667              23              23   
3  TCGA-AG-3742  chr1_55058667              19              19   
4  TCGA-AA-3663  chr1_55058667              19              21   

   allele_A_healthy  allele_B_healthy  period  ref repeat_id  \
0                18                19       2   26   1142870   
1                19                24       2   26   1142870   
2                19                19       2   26   1142870   
3                18                19       2   26   1142870   
4                19                19       2   26   1142870   

   patient_len_diff gene_name  beta_max  CAVIAR_score  
0                 5     USP24 -0.343976      0.017704  
1                 6     USP24 -0.343976      0.017704  
2                 8     USP24 -0.343976      0.017704  
3                 1     USP24 -0.343976      0.017704  
4                 2     USP24 -0.343976      0.017704

In [16]:
df_in_estr_variable.sort_values(by="CAVIAR_score", ascending=False).head()

patient         tmp_id  allele_A_tumor  allele_B_tumor  \
241  TCGA-AA-3673  chr3_40462030              13               8   
253  TCGA-AA-A02E  chr3_40462030              16               8   
230  TCGA-AA-3489  chr3_40462030              12              15   
231  TCGA-A6-2671  chr3_40462030              12              15   
232  TCGA-AG-3742  chr3_40462030              11              13   

     allele_A_healthy  allele_B_healthy  period  ref repeat_id  \
241                13                15       3   10    690639   
253                15                16       3   10    690639   
230                12                14       3   10    690639   
231                12                13       3   10    690639   
232                13                13       3   10    690639   

     patient_len_diff gene_name  beta_max  CAVIAR_score  
241                 7     RPL14 -0.361022      0.999949  
253                 7     RPL14 -0.361022      0.999949  
230                 1     RPL14 -0.361022      0.999949  
231                 2     RPL14 -0.361022      0.999949  
232                 2     RPL14 -0.361022      0.999949

In [17]:
df_in_estr_variable[df_in_estr_variable["CAVIAR_score"] > 0.3].sort_values(by="patient_len_diff", ascending=False)

patient          tmp_id  allele_A_tumor  allele_B_tumor  \
2780  TCGA-AA-3852  chr12_50086480              15              15   
2772  TCGA-AA-3679  chr12_50086480              14              14   
2768  TCGA-A6-5662  chr12_50086480               8               8   
2534  TCGA-AA-3856   chr6_32155564              25              30   
2536  TCGA-AA-3966   chr6_32155564              26              37   
...            ...             ...             ...             ...   
1818  TCGA-A6-3807  chr9_136980867              10              10   
1812  TCGA-AA-3712  chr9_136980867              11              11   
1758  TCGA-AA-A02W   chr6_33205972              14              15   
1755  TCGA-AA-3854   chr6_33205972              11              16   
1988  TCGA-AA-A00O   chr1_77711530              15              15   

      allele_A_healthy  allele_B_healthy  period  ref repeat_id  \
2780                 8                 8       3    8    563623   
2772                 8                 8       3    8    563623   
2768                14                14       3    8    563623   
2534                33                33       2   27     15145   
2536                27                27       2   27     15145   
...                ...               ...     ...  ...       ...   
1818                10                11       4    9   1547811   
1812                10                11       4    9   1547811   
1758                15                15       2   13     15529   
1755                11                15       2   13     15529   
1988                14                15       2   16   1156399   

      patient_len_diff gene_name  beta_max  CAVIAR_score  
2780                14      GPD1  0.262743      0.300894  
2772                12      GPD1  0.262743      0.300894  
2768                12      GPD1  0.262743      0.300894  
2534                11     PRRT1 -0.301305      0.597232  
2536                11     PRRT1 -0.301305      0.597232  
...                ...       ...       ...           ...  
1818                 1     FBXW5 -0.335319      0.304232  
1812                 1     FBXW5 -0.335319      0.304232  
1758                 1   COL11A2  0.309355      0.319064  
1755                 1   COL11A2  0.309355      0.319064  
1988                 1     USP33 -0.242324      0.421817  

[281 rows x 13 columns]

In [18]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

sns.scatterplot(
    data=df_in_estr_variable,
    x="patient_len_diff",
    y="beta_max",
    hue=df_in_estr_variable["CAVIAR_score"] > 0.3,
    s=100,
    ax=ax1
)
ax1.legend(title="fmSTR")

In [19]:
df_trusight_genes = pd.read_csv("/Users/maxverbiest/PhD/projects/genotype_strs/data/clinical/trusight_onco_genes.txt", sep="\t", header=None)
df_trusight_genes = df_trusight_genes.loc[:,[0,1]]
df_trusight_genes.columns = ["gene_id", "gene_name"]

df_trusight_genes["gene_id"] = df_trusight_genes.apply(lambda row: row.gene_id.split(".")[0], axis=1)
df_trusight_genes.head()

gene_id gene_name
0  ENSG00000097007      ABL1
1  ENSG00000143322      ABL2
2  ENSG00000115170     ACVR1
3  ENSG00000135503    ACVR1B
4  ENSG00000142208      AKT1

In [20]:
df_trusight_genes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 523 entries, 0 to 522
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   gene_id    523 non-null    object
 1   gene_name  523 non-null    object
dtypes: object(2)
memory usage: 8.3+ KB


In [21]:
df_in_estr

patient          tmp_id  allele_A_tumor  allele_B_tumor  \
13       TCGA-AA-3489    chr1_1752910              10              10   
459      TCGA-AA-3489   chr1_55058667              16              16   
497      TCGA-AA-3489   chr1_66643729               7               7   
646      TCGA-AA-3489  chr1_101021818              21              21   
673      TCGA-AA-3489  chr1_109759146              16              18   
...               ...             ...             ...             ...   
2105797  TCGA-AG-3890  chr19_32606027               6               6   
2106440  TCGA-AG-3890  chr5_176943071               4               4   
2106453  TCGA-AG-3890   chr5_34042762               6               6   
2106575  TCGA-AG-3890   chr6_26384925               3               3   
2106578  TCGA-AG-3890   chr6_29657256               7               7   

         allele_A_healthy  allele_B_healthy  period  ref repeat_id  \
13                     10                10       3    9   1100327   
459                    18                19       2   26   1142870   
497                     7                 7       2    7   1150529   
646                    22                22       2   21   1166924   
673                    15                17       2   19   1169780   
...                   ...               ...     ...  ...       ...   
2105797                 6                 6       4    5         .   
2106440                 4                 4       4    4         .   
2106453                 6                 6       3    5         .   
2106575                 3                 3       4    3         .   
2106578                 6                 6       3    7         .   

         patient_len_diff  
13                      0  
459                     5  
497                     0  
646                     2  
673                     2  
...                   ...  
2105797                 0  
2106440                 0  
2106453                 0  
2106575                 0  
2106578                 2  

[13658 rows x 10 columns]

In [29]:
pd.set_option("display.max_rows", 400)
pd.get_option("display.max_rows")

400

In [32]:
# df[df["gene_name"].isin(df_trusight_genes["gene_name"])].sort_values(by="CAVIAR_score", ascending=False).head(n=30)

# df_in_estr_variable.merge(df.loc[:, ["tmp_id", "gene_name", "beta_max", "CAVIAR_score"]], on="tmp_id", how="left").head()
df_in_estr_variable[df_in_estr_variable["gene_name"].isin(df_trusight_genes["gene_name"])].sort_values(by="patient_len_diff", ascending=False)

patient           tmp_id  allele_A_tumor  allele_B_tumor  \
2778  TCGA-AA-3852   chr12_50086480              15              15   
2770  TCGA-AA-3679   chr12_50086480              14              14   
2766  TCGA-A6-5662   chr12_50086480               8               8   
944   TCGA-A6-6653   chr17_42207488              23              23   
2782  TCGA-AA-3860   chr12_50086480              10              10   
2774  TCGA-AA-3696   chr12_50086480              15               6   
2605  TCGA-AD-A5EK   chr13_48316720              16              16   
3859  TCGA-AA-A00O   chr14_95116053              18              18   
2642  TCGA-AA-3678   chr2_201350682               7               7   
2601  TCGA-A6-5662   chr13_48316720              25              27   
2656  TCGA-AA-A00O   chr2_201350682               6               6   
3878  TCGA-AG-3890  chr11_108279460               3               3   
2636  TCGA-A6-4107   chr2_201350682               5               5   
3199  TCGA-AA-3812   chr5_132485829              14              14   
2654  TCGA-AA-3856   chr2_201350682               4               4   
3860  TCGA-AA-A017   chr14_95116053              17              17   
3877  TCGA-AA-A00O  chr11_108279460               3               3   
3874  TCGA-AA-3678  chr11_108279460               3               3   
4122  TCGA-AA-3814    chr3_10074917               4               4   
3241  TCGA-AA-A017   chr1_158611124               8               8   
3230  TCGA-AA-3672   chr1_158611124               6               6   
2429  TCGA-A6-5665   chr2_113195535               5              10   
545   TCGA-AA-3663   chr3_142375976              12              17   
3237  TCGA-AA-3856   chr1_158611124               6               6   
936   TCGA-A6-5665   chr17_42207488              15              15   
4073  TCGA-AA-3688   chr6_149773351               6               6   
958   TCGA-AA-3966   chr17_42207488              15              16   
961   TCGA-AA-A02W   chr17_42207488              20              20   
3861  TCGA-AA-A01K   chr14_95116053              19              19   
2648  TCGA-AA-3818   chr2_201350682               3               3   
3876  TCGA-AA-3814  chr11_108279460               4               4   
3667  TCGA-AA-3814   chr13_28068050               6               6   
937   TCGA-AZ-6598   chr17_42207488              14              16   
2600  TCGA-AA-3712   chr13_48316720              25              25   
2640  TCGA-AA-3672   chr2_201350682               5               5   
2634  TCGA-A6-3810   chr2_201350682               7               7   
3242  TCGA-AA-A01K   chr1_158611124               7               7   
2432  TCGA-AA-3693   chr2_113195535               5               5   
3239  TCGA-AA-3966   chr1_158611124               5               5   
563   TCGA-A6-6653   chr3_142375976              12              12   
3235  TCGA-AA-3818   chr1_158611124               4               4   
3233  TCGA-AA-3684   chr1_158611124               4               4   
573   TCGA-AA-3710   chr3_142375976              17              17   
2427  TCGA-AZ-6599   chr2_113195535               5               9   
3670  TCGA-AA-3860   chr13_28068050               6               6   
3187  TCGA-AA-3855    chr2_98544046               7               7   
1945  TCGA-A6-5665   chr5_180620740              11              11   
3672  TCGA-AA-A017   chr13_28068050               6               6   
3673  TCGA-AA-A01K   chr13_28068050               6               6   
3671  TCGA-AA-3875   chr13_28068050               4               4   
3666  TCGA-AA-3696   chr13_28068050               6               6   
3674  TCGA-AA-A02E   chr13_28068050               6               6   
963   TCGA-AD-6895   chr17_42207488              16              16   
3669  TCGA-AA-3854   chr13_28068050               6               6   
3662  TCGA-AA-3679   chr13_28068050               6               6   
3668  TCGA-AA-3848   chr13_28068050

In [38]:
df_in_estr_variable[df_in_estr_variable["gene_name"].isin(df_trusight_genes["gene_name"])].groupby("gene_name").size()
df_in_estr_variable[df_in_estr_variable["CAVIAR_score"] > 0.3].groupby("gene_name").size().sort_values(ascending=False).head(n=50)

gene_name
PRRT1        27
RPL14        27
SNX2         26
ASPRV1       22
COL11A2      20
S100A16      17
RBCK1        16
DHRS1        16
USP33        15
C10orf129    11
RAB7A        10
HSD17B4       9
FBXW5         9
FAH           7
SMPD1         7
SLC35B1       6
GPD1          5
RPL29         5
ANKDD1A       4
RNPEP         3
PCDHGA9       3
MARS2         3
ZNF165        3
KAT8          2
TRAFD1        2
PLEKHF2       1
SIKE1         1
IGFL4         1
UBE2B         1
HSPBP1        1
VPS36         1
dtype: int64

In [41]:
df_in_estr_variable[df_in_estr_variable["gene_name"] == "PRRT1"].sort_values(by="patient_len_diff", ascending=False)

patient         tmp_id  allele_A_tumor  allele_B_tumor  \
2536  TCGA-AA-3966  chr6_32155564              26              37   
2534  TCGA-AA-3856  chr6_32155564              25              30   
2533  TCGA-AA-3854  chr6_32155564              24              24   
2529  TCGA-AA-3814  chr6_32155564              24              24   
2538  TCGA-AA-A00O  chr6_32155564              28              28   
2522  TCGA-AA-3672  chr6_32155564              26              31   
2524  TCGA-AA-3678  chr6_32155564              18              18   
2532  TCGA-AA-3852  chr6_32155564              28              28   
2535  TCGA-AA-3949  chr6_32155564              23              23   
2516  TCGA-AA-3712  chr6_32155564              29              29   
2542  TCGA-AA-A02R  chr6_32155564              29              29   
2520  TCGA-A6-5660  chr6_32155564              18              24   
2527  TCGA-AA-3696  chr6_32155564              26              31   
2537  TCGA-AA-3968  chr6_32155564              23              29   
2540  TCGA-AA-A01K  chr6_32155564              26              26   
2539  TCGA-AA-A017  chr6_32155564              25              25   
2518  TCGA-A6-2675  chr6_32155564              27              27   
2523  TCGA-AA-3673  chr6_32155564              24              28   
2528  TCGA-AA-3812  chr6_32155564              27              27   
2541  TCGA-AA-A02E  chr6_32155564              27              27   
2526  TCGA-AA-3693  chr6_32155564              28              28   
2517  TCGA-AA-3662  chr6_32155564              28              28   
2521  TCGA-AA-3667  chr6_32155564              28              28   
2519  TCGA-A6-5659  chr6_32155564              19              24   
2525  TCGA-AA-3684  chr6_32155564              26              26   
2531  TCGA-AA-3848  chr6_32155564              18              30   
2530  TCGA-AA-3818  chr6_32155564              19              20   

      allele_A_healthy  allele_B_healthy  period  ref repeat_id  \
2536                27                27       2   27     15145   
2534                33                33       2   27     15145   
2533                29                29       2   27     15145   
2529                28                28       2   27     15145   
2538                25                33       2   27     15145   
2522                23                26       2   27     15145   
2524                18                26       2   27     15145   
2532                24                26       2   27     15145   
2535                26                26       2   27     15145   
2516                26                26       2   27     15145   
2542                26                26       2   27     15145   
2520                18                18       2   27     15145   
2527                27                27       2   27     15145   
2537                26                28       2   27     15145   
2540                28                28       2   27     15145   
2539                27                27       2   27     15145   
2518                25                25       2   27     15145   
2523                27                27       2   27     15145   
2528                26                29       2   27     15145   
2541                26                26       2   27     15145   
2526                29                29       2   27     15145   
2517                27                27       2   27     15145   
2521                27                27       2   27     15145   
2519                19                25       2   27     15145   
2525                25                26       2   27     15145   
2531                18                31       2   27     15145   
2530                19                19       2   27     15145   

      patient_len_diff gene_name  beta_max  CAVIAR_score  
2536                11     PRRT1 -0.301305      0.597232  
2534                11     PRRT1 -0.301305      0.597232  
2533                10  